In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
import openpyxl

from fastapi import FastAPI
from pydantic import BaseModel
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel

In [4]:
model = load_model("../../Downloads/model.h5")

In [5]:
train_df=pd.read_excel("C:\\MyData\\Downloads\\Telegram Desktop\\train_data.xlsx")
test_df=pd.read_excel("C:\\MyData\\Downloads\\Telegram Desktop\\test_data.xlsx")
X_train = train_df.iloc[:, :767]  
y_train = train_df['Label']     
X_test = test_df.iloc[:, :767]  
y_test = test_df['Label']     

In [10]:
# Load the pre-trained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased")

# Define unique labels
unique_labels = ['Заперечення', 'Виправдовування', 'Заклик', 
                 'Розпалювання ворожнечі та ненависті', 
                 'Приниження національної честі та гідності', 
                 'Просто текст']

label_to_id = {label: idx for idx, label in enumerate(unique_labels)}
id_to_label = {idx: label for label, idx in label_to_id.items()}

# Initialize FastAPI app
app = FastAPI()

# Request schema
class SentenceInput(BaseModel):
    text: str

def preprocess_text(text):
    import spacy  # Ensure spaCy is installed and the language model is loaded
    nlp = spacy.load("en_core_web_sm")  # Adjust to your language model
    doc = nlp(str(text).lower()) 
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return " ".join(tokens)

def get_avg_w2v(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    vector = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return vector

@app.post("/classify/")
def classify_sentence(sentence: SentenceInput):
    # Preprocess the input sentence
    processed_text = preprocess_text(sentence.text)
    
    # Convert to vector
    vector = get_avg_w2v(processed_text, tokenizer, model)
    
    # Simulating a classification step (replace with your model's prediction)
    # For example, use a simple nearest label algorithm based on predefined embeddings
    # Here, just a dummy random label is chosen for demonstration
    predicted_label_id = np.random.choice(list(label_to_id.values()))  # Replace with your classifier logic
    predicted_label = id_to_label[predicted_label_id]

    return {"text": sentence.text, "processed_text": processed_text, "label": predicted_label}



tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
